In [1]:
import ee

# Clear credentials
ee.Authenticate()


True

In [2]:
ee.Initialize()


In [3]:
import ee
import geemap

# Function to add a legend to the map
def add_legend_with_title(m, legend_title, legend_keys, legend_labels, legend_colors, position='bottomleft'):
    legend_dict = dict(zip(legend_keys, zip(legend_labels, legend_colors)))
    legend_html = f'<div style="text-align: left;"><strong>{legend_title}</strong></div><br>' + '<br>'.join([
        f'<span style="color: #{color}">{label} ({key})</span>' 
        for key, (label, color) in legend_dict.items()
    ])
    m.add_legend(legend_html, position=position)

# Initialize Earth Engine
ee.Initialize()

# Region of interest (Thiruvananthapuram district)
aoi = ee.FeatureCollection("projects/ee-prachoddileep/assets/Thiruvananthapuram1")

# MODIS NDVI data
dataset = ee.ImageCollection('MODIS/006/MOD13Q1') \
    .select('NDVI') \
    .filterBounds(aoi)

print(dataset.size().getInfo(), "Total number of NDVI Images")

# MODIS NDVI values come as NDVI x 10000 that need to be scaled by 0.0001
modisScaled = dataset.map(lambda image: image.divide(10000)
                          .copyProperties(image, ['system:index', 'system:time_start']))

# Months for which we require VCI
start_month = 1  # January
end_month = 1    # January

# Max and Min NDVI for all years but for the selected month
ndvi = modisScaled.filter(ee.Filter.calendarRange(start_month, end_month, 'month'))

# Add NDVI layers to the map
ndvi_max = ndvi.max().clip(aoi)
ndvi_min = ndvi.min().clip(aoi)

# Display NDVI_max and NDVI_min
m = geemap.Map()
m.centerObject(aoi, 8)
m.addLayer(ndvi_max, {}, 'NDVI_max', 0)
m.addLayer(ndvi_min, {}, 'NDVI_min', 0)

# Loop through years
for year in range(2007, 2013):  # Update the range as needed
    # NDVI for the specified month and year (mean instead of median)
    ndviT = modisScaled.filter(ee.Filter.calendarRange(year, year, 'year')) \
        .filter(ee.Filter.calendarRange(start_month, end_month, 'month')) \
        .mean() \
        .clip(aoi)

    # Display NDVIT layer
    m.addLayer(ndviT, {}, f'NDVIT_{year}', 0)

    # Final calculation of VCI for specified month & year
    VCI = ndviT.subtract(ndvi_min).divide(ndvi_max.subtract(ndvi_min)).rename('VCI')

    # Visualization parameters for VCI layer (customize the palette)
    VCIviz_custom = {'min': 0, 'max': 1, 'palette': ['FF0000', 'FFFF00', '00FF00', '0000FF']}
    
    # Add VCI layer to the map with customized visualization parameters
    m.addLayer(VCI, VCIviz_custom, f'VCI_{year}', 1)

# Add custom legend with title
legend_title = 'Drought Condition'
legend_keys = ['0-0.25', '0.25-0.5', '0.5-0.75', '0.75-1.0']
legend_labels = ['High', 'Moderate', 'Low', 'Very Low']
legend_colors = ['FF0000', 'FFFF00', '00FF00', '0000FF']
add_legend_with_title(m, legend_title, legend_keys, legend_labels, legend_colors, position='bottomleft')

# All VCI at once
def compute_vci(img):
    img = img.clip(aoi)
    date = ee.Date(img.get('system:time_start'))
    month = date.get('month')
    dataset2 = modisScaled.filter(ee.Filter.calendarRange(month, month, 'month'))
    NDVI_min = dataset2.min()
    NDVI_max = dataset2.max()
    VCI = img.subtract(NDVI_min).divide(NDVI_max.subtract(NDVI_min)).rename('VCI') \
        .copyProperties(img, ['system:time_start'])
    return VCI

modVCI = modisScaled.map(compute_vci)

# Add VCI_IC layer to the map with the same visualization parameters
m.addLayer(modVCI, VCIviz_custom, 'VCI_IC', 1)

# Add layer control
m.add_layer_control()

# Center the map on the region of interest
m.centerObject(aoi, 8)

# Save the map as an HTML file
output_html = r"D:\Drought Project\map123\output_map.html"
m.save(outfile=output_html)

# Display the map
m


C:\Users\91954\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


529 Total number of NDVI Images


Map(center=[8.608436183893986, 77.01503663739864], controls=(WidgetControl(options=['position', 'transparent_b…

In [4]:
import ee

# Initialize Earth Engine
ee.Initialize()

# Region of interest (Bundelkhand district)
aoi = ee.FeatureCollection("projects/ee-prachoddileep/assets/Thiruvananthapuram1")

# MODIS NDVI data
dataset = ee.ImageCollection('MODIS/006/MOD13Q1') \
    .select('NDVI') \
    .filterBounds(aoi)

# Create a list to store the number of images for each month
images_per_month = []

# Loop through each month
for month in range(1, 13):  # January to December
    # Filter dataset for the current month
    ndvi_month = dataset.filter(ee.Filter.calendarRange(month, month, 'month'))
    
    # Get the size of the filtered dataset and append it to the list
    images_per_month.append(ndvi_month.size().getInfo())

# Print the number of NDVI images for each month
for i, count in enumerate(images_per_month, start=1):
    print(f"Month {i}: {count} NDVI Images")


Month 1: 46 NDVI Images
Month 2: 46 NDVI Images
Month 3: 46 NDVI Images
Month 4: 46 NDVI Images
Month 5: 46 NDVI Images
Month 6: 46 NDVI Images
Month 7: 46 NDVI Images
Month 8: 46 NDVI Images
Month 9: 46 NDVI Images
Month 10: 29 NDVI Images
Month 11: 40 NDVI Images
Month 12: 46 NDVI Images


In [7]:
import ee
import geemap

# Function to add a legend to the map
def add_legend_with_title(m, legend_title, legend_keys, legend_labels, legend_colors, position='bottomleft'):
    legend_dict = dict(zip(legend_keys, zip(legend_labels, legend_colors)))
    legend_html = f'<div style="text-align: left;"><strong>{legend_title}</strong></div><br>' + '<br>'.join([
        f'<span style="color: #{color}">{label} ({key})</span>' 
        for key, (label, color) in legend_dict.items()
    ])
    m.add_legend(legend_html, position=position)

# Initialize Earth Engine
ee.Initialize()

# Region of interest (Thiruvananthapuram district)
aoi = ee.FeatureCollection("projects/ee-prachoddileep/assets/Thiruvananthapuram1")

# MODIS NDVI data
dataset = ee.ImageCollection('MODIS/006/MOD13Q1') \
    .select('NDVI') \
    .filterBounds(aoi)

print(dataset.size().getInfo(), "Total number of NDVI Images")

# MODIS NDVI values come as NDVI x 10000 that need to be scaled by 0.0001
modisScaled = dataset.map(lambda image: image.divide(10000)
                          .copyProperties(image, ['system:index', 'system:time_start']))

# Months for which we require VCI
start_month = 1  # January
end_month = 12   # December

# Filter data for the year 2022
ndvi_2022 = modisScaled.filter(ee.Filter.calendarRange(2022, 2022, 'year'))

# Add NDVI layers to the map for each month of 2022
m = geemap.Map()
m.centerObject(aoi, 8)

# Loop through all months
for month in range(1, 13):  # January to December
    # NDVI for the specified month and year (mean instead of median)
    ndviT = ndvi_2022.filter(ee.Filter.calendarRange(month, month, 'month')) \
        .mean() \
        .clip(aoi)

    # Display NDVIT layer
    m.addLayer(ndviT, {}, f'NDVIT_{month}', 0)

    # Final calculation of VCI for specified month & year
    ndvi_min = modisScaled.filter(ee.Filter.calendarRange(month, month, 'month')).min()
    ndvi_max = modisScaled.filter(ee.Filter.calendarRange(month, month, 'month')).max()
    VCI = ndviT.subtract(ndvi_min).divide(ndvi_max.subtract(ndvi_min)).rename('VCI')

    # Visualization parameters for VCI layer (customize the palette)
    VCIviz_custom = {'min': 0, 'max': 1, 'palette': ['FF0000', 'FFFF00', '00FF00', '0000FF']}
    
    # Add VCI layer to the map with customized visualization parameters
    m.addLayer(VCI, VCIviz_custom, f'VCI_{month}', 1)

# Add custom legend with title
legend_title = 'Drought Condition'
legend_keys = ['0-0.25', '0.25-0.5', '0.5-0.75', '0.75-1.0']
legend_labels = ['High', 'Moderate', 'Low', 'Very Low']
legend_colors = ['FF0000', 'FFFF00', '00FF00', '0000FF']
add_legend_with_title(m, legend_title, legend_keys, legend_labels, legend_colors, position='bottomleft')

# Add layer control
m.add_layer_control()

# Save the map as an HTML file
output_html = r"D:\Drought Project\map123\output_map_2022.html"
m.save(outfile=output_html)

# Display the map
m


529 Total number of NDVI Images


Map(center=[8.608436183893986, 77.01503663739864], controls=(WidgetControl(options=['position', 'transparent_b…